In [41]:
import requests
from bs4 import BeautifulSoup
import json
import requests
import urllib.parse
import urllib3
import concurrent.futures
import time

# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)



In [42]:


# Danh sách các thương hiệu (brand array) cần kiểm tra
brand_array = [
    "111SKIN", "19/99 Beauty", "ABOUT-FACE", "Acqua Di Parma", "Act+Acre", 
    "Advanced Nutrition Programme", "adwoa beauty", "Aesop", "Alfresco", "Algenist", 
    "Allies of Skin", "Alpha-H", "Ameliorate", "Amika", "Anastasia Beverly Hills", 
    "Augustinus Bader", "Aveda", "Avène", "bareMinerals", "BEAUTYBLENDER", "Bed of Nails", 
    "Bella Freud", "Benefit", "BIOEFFECT", "Biossance", "BLEACH London", "Bondi Sands", 
    "Bouclème", "Boy Smells", "BREAD BEAUTY SUPPLY", "Briogeo", "BY TERRY", "BYOMA", "BYREDO", 
    "Caudalie", "CeraVe", "Ceremonia", "Chantecaille", "Charlotte Tilbury", "Christophe Robin", 
    "Clé de Peau Beauté", "Coco & Eve", "COCO DE MER", "Codex Beauty Labs", "Color Wow", 
    "Comme Des Garçons", "Community Sixty-Six", "Conserving Beauty", "COOLA", "CORPUS", 
    "COSRX", "CRA-YON", "Creed", "Cult Beauty", "Cultured", "Current State", "D.S. & Durga", 
    "Danessa Myricks Beauty", "DARLING", "Davines", "de Mamiel", "Decree", "Déesse Pro", "DeoDoc", 
    "Dermalogica", "Dieux Skin", "Discothèque", "Doré", "dpHUE", "Dr. Barbara Sturm", "Dr. Dana", 
    "Dr. Dennis Gross Skincare", "Dr. Jart+", "Dr. LEVY Switzerland", "Drowsy", "Drunk Elephant", 
    "Drybar", "Earl of East", "EDY LONDON", "Elemis", "Embryolisse", "Emma Hardie Skincare", 
    "Escentric Molecules", "ESPA Skincare", "EVY Technology", "EXA", "Eyeko", "Fable & Mane", 
    "FaceGym", "FARMACY", "Fekkai", "Fig.1 Beauty", "FOREO", "frank body", "Frédéric Malle", "Fresh", 
    "Fugazzi Fragrances", "Function of Beauty", "Fur", "Gallinée", "ghd", "Gisou", "Glow Recipe", 
    "good light", "Grow Gorgeous", "Grown Alchemist", "Guerlain", "Gun Ana", "Hair by Sam McKnight", 
    "HAIR GAIN", "Hair Rituel", "HELLO KLEAN", "Hello Sunday", "Herbivore", "Herlum", "Hermès", 
    "High Beauty", "Higherdose", "HigherDOSE", "HINDASH", "Holifrog", "Hourglass", "Huda Beauty", 
    "HUM Nutrition", "Humanrace", "ICONIC LONDON", "ilāpothecary", "Innersense", "Innisfree", 
    "ISAMAYA", "Isle of Paradise", "IT Cosmetics", "Jo Loves", "Jo Malone London", 
    "Jordan Samuel Skin", "Jouer Cosmetics", "JVN Hair", "K18", "Kate Somerville", "KEVIN.MURPHY", 
    "Kevyn Aucoin", "Kiehl's", "Kilian Perfume", "Kinship", "Kitsch", "Kopari Beauty", "KORA Organics", 
    "Kosas", "Krave beauty", "L'Occitane", "La Bouche Rouge Paris", "La Mer", "La Prairie", 
    "La Roche-Posay", "Laboratory Perfumes", "Lancaster Beauty", "LANEIGE", "Lanolips", "Larry King Hair Care", 
    "Laura Mercier", "Le Labo", "LELO", "LGBTQIA+  Owned Businesses", "Live Tinted", "Living Proof", 
    "Loving Tan", "Luna Daily", "Lyma", "LYS Beauty", "Maison Margiela", "Make Up For Ever", 
    "MALIN + GOETZ", "Mama Mio", "Manifesto", "Manucurist", "Mario Badescu", "Marvis", "Medik8", "Megababe", 
    "Melanin Haircare", "MEMO PARIS", "MILK MAKEUP", "MINI BLOOM", "Mio Skincare", "MOA - Magic Organic Apothecary", 
    "Molton Brown", "Monika Blunder Beauty", "Monpure", "MPowder", "Muihood", "Murad", "MUTHA", "Myvitamins", 
    "MZ Skin", "NAILS.INC", "NARS", "Natasha Denona", "Naturally Drenched", "NatureLab Tokyo", "NAYDAYA", 
    "Neen", "NEST New York", "NIOD", "Nudestix", "NuFACE", "Nurse Jamie", "Nuxe", "NYX Professional Makeup", 
    "Odacite", "OLAPLEX", "OLE HENRIKSEN", "Omorovicza", "Oribe", "Oskia", "OUAI", "OVEROSE", "PACIFICA", 
    "Pai Skincare", "Pat McGrath", "Patchology", "PATRICK TA", "Pattern", "Paula's Choice", "Peace Out", 
    "Perricone MD", "Peter Thomas Roth", "Philip Kingsley", "Pixi", "PolarSEAL", "PSA", "Qure Skincare", 
    "R+Co", "Rae Morris", "RANAVAT", "Rejuvenated Ltd", "REN Clean Skincare", "Resorè", "RevitaLash", 
    "Rituel de Fille", "RMS BEAUTY", "RÓEN", "Rose Inc", "Royal Fern", "Sachajuan", "Saie", 
    "Sana Jardin", "Sarah Chapman", "Shani Darden Skin Care", "Shiseido", "Silk'n", "SILKE London", 
    "SIMIHAZE BEAUTY", "Sisley-Paris", "Slip", "Smile Makers", "Sol de Janeiro", "Solaris Labs NY", "Solgar", 
    "Spacemasks", "Spectacle Skincare", "Spectrum Collections", "St. Tropez", "SUMMER FRIDAYS", "Sun Bum", 
    "Sundae Body", "Sunday Riley", "Supergoop!", "SUQQU", "Surratt", "Susanne Kaufmann", "Sweed", "Tan-Luxe", 
    "Tata Harper", "Thank You Farmer", "The Beauty Chef", "THE INKEY LIST", "The Light Salon", "The Nue Co.", 
    "The Ordinary", "The Outset", "The Perfumer's Story by Azzi", "The Seated Queen", "Then I Met You", 
    "Too Faced", "Tower 28", "TRUDON", "True Botanicals", "Tweezerman", "U Beauty", "Ule Beauty", 
    "Ultra Violette", "Uma", "Urban Decay", "Vanity Makeup", "VERDEN", "Vida Glow", "VIEVE", 
    "Vintner's Daughter", "Virtue", "Vital Proteins", "VOTARY", "Vyrao", "Weleda", "WelleCo", 
    "Westman Atelier", "WetBrush", "WISHFUL", "Youth To The People", "Zelens", "ZITSTICKA"
]


# Hàm để crawl dữ liệu từ một trang cụ thể
def crawl_page(page_number):
    url = f"https://www.cultbeauty.com/search/?q=&pageNumber={page_number}"
    response = requests.get(url)
    
    # Kiểm tra nếu phản hồi HTTP là thành công
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to retrieve page {page_number}")
        return None



In [43]:
def send_to_google_forms(product):
    # URL Google Form
    url = "https://docs.google.com/forms/d/e/1FAIpQLSeeuY046qewnd0Dxk5Rz8ok30OyI3zsDmeeAXkhZSJouu62eQ/formResponse"

    # Các trường dữ liệu của Google Form (đảm bảo đã thay thế đúng các entry ID trong form của bạn)
    form_data = {
        'entry.690943670': product.get('product_name', ''),
        'entry.1818637395': product.get('brand', ''),
        'entry.1892748356': product.get('sale_price', ''),
        'entry.1721193677': product.get('sale_off', ''),
        'entry.158181074': product.get('product_url', ''),
        'entry.986500295': product.get('img_url', ''),
        'fvv': '1'
    }

    # Gửi POST request vào Google Form
    try:
        response = requests.post(url, data=form_data)
        
        # Kiểm tra xem request có thành công không
        if response.status_code == 200:
            print("Data sent successfully!")
        else:
            print(f"Failed to send data. Status code: {response.status_code}")
    except Exception as e:
        print(f"Error occurred while sending data: {str(e)}")


In [44]:
# Hàm sử dụng ThreadPoolExecutor để gửi dữ liệu song song
def send_products_in_parallel(products):
    # Sử dụng ThreadPoolExecutor để gửi dữ liệu song song
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Executor sẽ gửi từng sản phẩm lên Google Form
        executor.map(send_to_google_forms, products)

In [45]:
# Hàm trích xuất thông tin sản phẩm từ HTML
def extract_product_info(soup):
    products = []

    # Tìm tất cả các sản phẩm dựa trên data-e2e
    for i in range(1, 30):
        product = {}

        # Tìm thẻ chứa thông tin sản phẩm
        product_item = soup.find(attrs={"data-e2e": f"search_list-item-{i}"})
        if not product_item:
            continue

        # Lấy product_name từ thẻ <a class="product-item-title">
        product_name_tag = product_item.find('a', {'aria-hidden': 'true'})
        if product_name_tag:
            product['product_name'] = product_name_tag.get_text(strip=True)
            product['product_url'] = "https://www.cultbeauty.com" + product_name_tag['href']

        # Lấy sale_off (giảm giá)
        sale_off_tag = product_item.find('span', {'class': 'flex items-center gap-2 justify-center'})
        if sale_off_tag:
            sale_off_text = sale_off_tag.get_text(strip=True)
            sale_off_match = re.search(r'(\d+)%', sale_off_text)  # Lấy phần số và %
            if sale_off_match:
                product['sale_off'] = sale_off_match.group(1) + '%'
            else:
                product['sale_off'] = "No Sale"

        # Lấy regular_price từ thẻ <s>
        regular_price_tag = product_item.find('s', {'class': 'mt-6 mr-1'})
        if regular_price_tag:
            regular_price = regular_price_tag.get_text(strip=True)
            # Chuyển đổi ký tự bị lỗi sang ký tự Euro chuẩn (dùng phương thức replace)
            regular_price = regular_price.replace("â\x82¬", "€")
            product['regular_price'] = regular_price

        # Lấy sale_price từ thẻ <span class="text-gray-900">, nếu không có thì lấy regular_price
        sale_price_tag = product_item.find('span', {'class': 'text-gray-900'})
        if sale_price_tag:
            sale_price = sale_price_tag.get_text(strip=True)
            # Chuyển đổi ký tự bị lỗi sang ký tự Euro chuẩn (dùng phương thức replace)
            sale_price = sale_price.replace("â\x82¬", "€")
            product['sale_price'] = sale_price
        else:
            # Nếu không tìm thấy sale_price, dùng regular_price làm sale_price
            product['sale_price'] = regular_price  
            
        # Lấy img_url từ thẻ <img src>
        img_tag = product_item.find('img')
        if img_tag and 'src' in img_tag.attrs:
            product['img_url'] = img_tag['src']

        # Kiểm tra brand trong product_name
        product_name = product_item.find('a', {'class': 'product-item-title'}).get_text(strip=True)
        product['product_name'] = product_name  # Gán tên sản phẩm vào product

        # Tìm thương hiệu trong product_name từ brand_array
        product['brand'] = None
        for brand in brand_array:
            if brand.lower() in product_name.lower():  # So sánh không phân biệt hoa thường
                product['brand'] = brand
                break

        # Nếu không tìm thấy thương hiệu, lấy từ đầu tiên trong product_name
        if not product['brand']:
            product['brand'] = product_name.split()[0]

        # Capture các ký hiệu tiền tệ Euro (€) và Dollar ($)
        currency_pattern = r'(\€|\$|\bEUR\b|\bUSD\b)'
        currency_matches = re.findall(currency_pattern, product_name)
        
        # Lưu sản phẩm vào danh sách
        if product:
            products.append(product)

    return products



In [ ]:
# Crawl dữ liệu từ trang 1 đến 2 và gửi vào Google Forms
all_products = []
for i in range(1, 2):  # Crawl từ trang 1 đến 2
    page_html = crawl_page(i)
    if page_html:
        soup = BeautifulSoup(page_html, 'html.parser')
        products = extract_product_info(soup)
        all_products.extend(products)

# Gửi kết quả lên Google Forms và in ra kết quả
for product in all_products:
    send_products_in_parallel(products)  # Gửi từng sản phẩm vào Google Form
    print("Succes Sent!")  # In thông tin sản phẩm
    

Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Succes Sent!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Data sent successfully!
Dat

KeyboardInterrupt: 

Data sent successfully!
